In [ ]:
!git clone https://github.com/thanhtlx/deepseek-eval
!pip install -r deepseek-eval/requirements.txt > tmp
!pip install pandas fire > tmp
import os
os.environ['TOKENIZERS_PARALLELISM']='false'
os.chdir('deepseek-eval/Evaluation/HumanEval')
!mkdir tmp_dir

In [ ]:
!cd ../..  && git pull

In [ ]:
from utils.dataset import HumanEvalDataset
data_root ='data/'
language = 'python'
dataset = HumanEvalDataset(data_root, language=language)


GENERATE CODE

In [ ]:
from tqdm import tqdm
result = list()
PROMPT_TEMPLATE = """
I want you to act as a program developer.
Do not write any explanations or other words, just reply with the complete function.
%s
"""
queue_api = list()
for idx in tqdm(range(0, len(dataset))):
    data = dataset[idx]
    prompt = PROMPT_TEMPLATE.strip() % data['prompt']
    post_api = {"custom_id":data['task_id'],
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                "model": "gpt-3.5-turbo",
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0,
                "max_tokens": 256}}
    queue_api.append(post_api)
    print(data['prompt'])
    print("------------")
import json
with open('queue_api_xai.jsonl','w+') as f:
    for obj in queue_api:
        f.writelines(json.dumps(obj)+"\n")

GERERATE DEPENDENCE

In [53]:

# xxx
import pandas as pd 
import json 

file = 'Human_eval_chatgpt_java.csv'
lang = 'java'
df = pd.read_csv(file)
result_data = list()
result_control = list()

def get_post_apt(source_content,type_dependence,task_id):
    prompt = f"List all pairs of statements by line numbers which have {type_dependence} relationship in the given source code: \n""" + source_content
    post = {"custom_id":task_id,
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                "model": "gpt-3.5-turbo-0125",
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0,
                "max_tokens": 512}}
    return post

for _,row in df.iterrows():
    source_content = row['function']
    task_id = row['task_id']+"#####data_dep"
    result_data.append(get_post_apt(source_content,"data dependence",task_id))
    task_id = row['task_id']+"#####control_dep"
    result_control.append(get_post_apt(source_content,"control dependence",task_id))

with open(f'human_eval\\{lang}\\'+f"{lang}_data_dep.jsonl",'w+') as f:
    for obj in result_data:
        f.writelines(json.dumps(obj) + '\n')
with open(f'human_eval\\{lang}\\'+f"{lang}_control_dep.jsonl",'w+') as f:
    for obj in result_control:
        f.writelines(json.dumps(obj) + '\n')

GENERATE NAME PRD

In [55]:

# xxx
import pandas as pd 
import json
from utils2 import get_node_edges
file = 'Human_eval_chatgpt_python.csv'
lang = 'python'
file = 'Human_eval_chatgpt_java.csv'
lang = 'java'
df = pd.read_csv(file)
result = list()

def get_post_apt(source_content,index):
    prompt = "Generate the function name for the given function: \n""" + source_content
    post = {"custom_id":index,
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                "model": "gpt-3.5-turbo-0125",
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0,
                "max_tokens": 256}}
    return post
fields = ['function','RenameVariable','ForWhileConvert','ReverseIf','ReorderParameter','ChangeCondition','RemoveConditionStatement','ChangeOperator','RemoveDefStatement']
import re
for idx,row in df.iterrows():
    node_content = df.at[idx, "nodes"]
    edge_content = df.at[idx, "edges"]
    source_content = row['function']
    method_name = ""
    try:
        if lang == 'java':
            nodes, edges = get_node_edges(source_content, edge_content, node_content)
            nodes = nodes[nodes["_label"] == "METHOD"]
            method_name = nodes[nodes["code"].str.contains("public static")]["name"].tolist()[0]
        else:
            nodes = json.loads(row['nodes'])
            tmp = [n for n in nodes if n["_label"] == "METHOD" and '<' not in n['name']]
            if len(tmp) > 0:
                method_name = tmp[0]['name']
            else:
                method_name = '<UKN>'
        
    except Exception as e:
        print(row['task_id'],e)
        if lang == "java":
            for lineCode in source_content.splitlines():
                if 'public static' in lineCode:
                    res = re.findall(r'\s([A-z0-9_]+)\(',lineCode)
                    if len(res) > 0 :
                        method_name = res[0]
                        print(method_name)
                        break
        if method_name == "":
            continue
        
    for field_name in fields:
        source_content = df.at[idx, field_name].replace(method_name, "<FILL>")
        index = f'{row["task_id"]}#####{method_name}#####{field_name}'
        result.append(get_post_apt(source_content,index))

with open(f'human_eval\\{lang}\\'+f"{lang}_method_name.jsonl",'w+') as f:
    for obj in result:
        f.writelines(json.dumps(obj) + '\n')

"['lineNumber'] not in index"
HumanEval_129_minPath cannot unpack non-iterable NoneType object
minPath
"['lineNumber'] not in index"
HumanEval_39_prime_fib cannot unpack non-iterable NoneType object
primeFib
"['lineNumber'] not in index"
HumanEval_117_select_words cannot unpack non-iterable NoneType object
selectWords
"['lineNumber'] not in index"
HumanEval_68_pluck cannot unpack non-iterable NoneType object
pluck
HumanEval_147_get_max_triples Incorrect Type Assumption.
getMaxTriples
"['lineNumber'] not in index"
HumanEval_47_median cannot unpack non-iterable NoneType object
median
"['lineNumber'] not in index"
HumanEval_127_intersection cannot unpack non-iterable NoneType object
intersection
"['lineNumber'] not in index"
HumanEval_152_compare cannot unpack non-iterable NoneType object
compare
"['lineNumber'] not in index"
HumanEval_135_can_arrange cannot unpack non-iterable NoneType object
canArrange
"['lineNumber'] not in index"
HumanEval_149_sorted_list_sum cannot unpack non-iterabl

GENERATE SUMMARY

In [56]:

# xxx
import pandas as pd 
from utils2 import get_node_edges
file = 'Human_eval_chatgpt_python.csv'
lang = 'python'
file = 'Human_eval_chatgpt_java.csv'
lang = 'java'
df = pd.read_csv(file)
result = list()

def get_post_apt(source_content,index):
    prompt = "Summarize the given source code: \n""" + source_content
    post = {"custom_id":index,
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                "model": "gpt-3.5-turbo-0125",
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0,
                "max_tokens": 512}}
    return post
fields = ['function','RenameVariable','ForWhileConvert','ReverseIf','ReorderParameter','ChangeCondition','RemoveConditionStatement','ChangeOperator','RemoveDefStatement']

for idx,row in df.iterrows():
    for field_name in fields:
        source_content = df.at[idx, field_name]
        index = f'{row["task_id"]}#####{field_name}'
        result.append(get_post_apt(source_content,index))

with open(f'human_eval\\{lang}\\'+f"{lang}_summarize.jsonl",'w+') as f:
    for obj in result:
        f.writelines(json.dumps(obj) + '\n')

GENERATE OUTPUT

In [57]:

# xxx
import pandas as pd 
from utils2 import get_node_edges
from redbaron import RedBaron

file = 'Human_eval_chatgpt_python.csv'
lang = 'python'
file = 'Human_eval_chatgpt_java.csv'
lang = 'java'
df = pd.read_csv(file)
result = list()

def get_post_apt(source_content,tin,index):
    prompt =  """Given the function: \n""" + source_content + "\n Please complete the following test case: " + tin +":"
    post = {"custom_id":index,
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                "model": "gpt-3.5-turbo-0125",
                "messages": [{"role": "user", "content": prompt}],
                "temperature": 0,
                "max_tokens": 256}}
    return post
fields = ['function','RenameVariable','ForWhileConvert','ReverseIf','ReorderParameter','ChangeCondition','RemoveConditionStatement','ChangeOperator','RemoveDefStatement']

for idx,row in df.iterrows():
    source_content = row['function']
    code = row['code']
    test_cases = list()
    for line in code.splitlines():
        if lang == "java":
        
            if line.strip().startswith("assert"):
                # print(line)
                if "==" in line:
                    testinput = line.split("==")[0]
                    testoutput = line.split("==")[1]
                elif "equals" in line:
                    testinput = line.split(".equals")[0]
                    testoutput = line.split(".equals")[1]
                test_cases.append((line.replace(testoutput, " <FILL> "), testoutput))
        else:
            if line.strip().startswith("assert"):
                try:
                    red = RedBaron(line.strip())
                    testinput = red[0].value.first.dumps()
                    testoutput = red[0].value.second.dumps()
                    test_cases.append((line.replace(testoutput, " <FILL> "), testoutput))
                except Exception as e:
                    print(e) 
    for field_name in fields:
        source_content = df.at[idx, field_name].replace(method_name, "<FILL>")
        for (tin, tout) in test_cases:
            index = f'{row["task_id"]}#####{tin}#####{tout}#####{field_name}'
            result.append(get_post_apt(source_content,tin,index))

with open(f'human_eval\\{lang}\\'+f"{lang}_output.jsonl",'w+') as f:
    for obj in result:
        f.writelines(json.dumps(obj) + '\n')

In [61]:
import glob 
import json 
for file in glob.glob("human_eval\\open_ai_call\\mini_batch*.jsonl"):
    print(file)
    result = list()
    mapping = dict()
    with open(file) as f:
        idx = 0
        for line in f.readlines():
            idx += 1
            obj = json.loads(line.strip())
            mapping[str(idx)] = obj['custom_id']
            obj['custom_id'] = str(idx)
            result.append(obj)
    with open(file,'w+') as ff:
        for obj in result:
            ff.writelines(json.dumps(obj) +'\n')
    with open(file+".mapping.json",'w+') as f:
        json.dump(mapping,f)
    

human_eval\open_ai_call\mini_batch_0_java_control_dep.jsonl
human_eval\open_ai_call\mini_batch_0_java_data_dep.jsonl
human_eval\open_ai_call\mini_batch_0_java_method_name.jsonl
human_eval\open_ai_call\mini_batch_0_java_output.jsonl
human_eval\open_ai_call\mini_batch_0_java_summarize.jsonl
human_eval\open_ai_call\mini_batch_0_python_control_dep.jsonl
human_eval\open_ai_call\mini_batch_0_python_data_dep.jsonl
human_eval\open_ai_call\mini_batch_0_python_method_name.jsonl
human_eval\open_ai_call\mini_batch_0_python_output.jsonl
human_eval\open_ai_call\mini_batch_0_python_summarize.jsonl
human_eval\open_ai_call\mini_batch_10_java_output.jsonl
human_eval\open_ai_call\mini_batch_11_java_output.jsonl
human_eval\open_ai_call\mini_batch_12_java_output.jsonl
human_eval\open_ai_call\mini_batch_13_java_output.jsonl
human_eval\open_ai_call\mini_batch_1_java_method_name.jsonl
human_eval\open_ai_call\mini_batch_1_java_output.jsonl
human_eval\open_ai_call\mini_batch_1_java_summarize.jsonl
human_eval\op